In [ ]:
from manipurl.utils.embedding_buffer import EmbeddingBuffer
import torch

buffer = EmbeddingBuffer(4, 10)

print(buffer.pos_buffer)
print(buffer.pos_ptr)
print(buffer.pos_part)

buffer.insert_episode(torch.ones((3, 4)) * 1, True)

print(buffer.pos_buffer)
print(buffer.pos_ptr)
print(buffer.pos_part)

buffer.insert_episode(torch.ones((3, 4)) * 2, True)

print(buffer.pos_buffer)
print(buffer.pos_ptr)
print(buffer.pos_part)

buffer.insert_episode(torch.ones((3, 4)) * 3, True)

print(buffer.pos_buffer)
print(buffer.pos_ptr)
print(buffer.pos_part)

buffer.insert_episode(torch.ones((3, 4)) * 4, True)

print(buffer.pos_buffer)
print(buffer.pos_ptr)
print(buffer.pos_part)

In [4]:
test = torch.zeros(5)

test[:2] = 1
print(test)

tensor([1., 1., 0., 0., 0.])


In [13]:
print(buffer.pos_part)
gap = 2
print(buffer.pos_part[((torch.arange(len(buffer.pos_part)) - gap) % len(buffer.pos_part))])
print(buffer.pos_part != buffer.pos_part[((torch.arange(len(buffer.pos_part)) - gap) % len(buffer.pos_part))])

tensor([4., 4., 1., 2., 2., 2., 3., 3., 3., 4.], device='cuda:0')
tensor([3., 4., 4., 4., 1., 2., 2., 2., 3., 3.], device='cuda:0')
tensor([ True, False,  True,  True,  True, False,  True,  True, False,  True],
       device='cuda:0')


In [14]:
torch.where(buffer.pos_part == 4)

(tensor([0, 1, 9], device='cuda:0'),)